- 전처리 과정이 납득이 잘 되지 않음
  - https://www.bigdata-region.kr/#/dataset/0ad3c882-f7ee-4faf-970d-00c53cb65a84
  - 위 링크를 참고하여, 데이터 상 격자공간고유번호가 격자공간50미터 기준인 것으로 파악
  - 격자공간 50미터의 경우, 시군구코드 + 나머지 형식으로 이루어짐
  - 따라서 격자공간고유번호를 위와 같이 두 개로 나누기로 결정

In [1]:
!pip install catboost
!pip install optuna

In [7]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor

from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error

import random
import optuna
from optuna.samplers import TPESampler

import warnings
warnings.filterwarnings("ignore")

In [37]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

submission = pd.read_csv('./data/sample_submission.csv')

train.columns = ['index', 'SEND_SPG_INNB', 'REC_SPG_INNB', 'product_category', 'INVC_CONT']
test.columns = ['index', 'SEND_SPG_INNB', 'REC_SPG_INNB', 'product_category']

train.index=train['index']
test.index=test['index']

train.drop("index", axis = 1, inplace = True)
test.drop("index", axis = 1, inplace = True)

# 격자공간고유번호 나누기 
# 앞 5개 region_code 열에 저장, 나머지 space_number 열에 저장

# train, SEND_SPG_INNB
li_region_code = []
li_space_number = []

for i in range(len(train)):
    li_region_code.append(int(str(train["SEND_SPG_INNB"][i])[0:5]))
    li_space_number.append(int(str(train["SEND_SPG_INNB"][i])[5:]))
    
train["SEND_region_code"] = li_region_code
train["SEND_space_number"] = li_space_number

train.drop("SEND_SPG_INNB", axis = 1, inplace = True)


# train, REC_SPG_INNB
li_region_code = []
li_space_number = []

for i in range(len(train)):
    li_region_code.append(int(str(train["REC_SPG_INNB"][i])[0:5]))
    li_space_number.append(int(str(train["REC_SPG_INNB"][i])[5:]))
    
train["REC_region_code"] = li_region_code
train["REC_space_number"] = li_space_number

train.drop("REC_SPG_INNB", axis = 1, inplace = True)


# test, SEND_SPG_INNB
li_region_code = []
li_space_number = []

for i in range(len(test)):
    li_region_code.append(int(str(test["SEND_SPG_INNB"][i])[0:5]))
    li_space_number.append(int(str(test["SEND_SPG_INNB"][i])[5:]))
    
test["SEND_region_code"] = li_region_code
test["SEND_space_number"] = li_space_number

test.drop("SEND_SPG_INNB", axis = 1, inplace = True)


# test, REC_SPG_INNB
li_region_code = []
li_space_number = []

for i in range(len(test)):
    li_region_code.append(int(str(test["REC_SPG_INNB"][i])[0:5]))
    li_space_number.append(int(str(test["REC_SPG_INNB"][i])[5:]))
    
test["REC_region_code"] = li_region_code
test["REC_space_number"] = li_space_number

test.drop("REC_SPG_INNB", axis = 1, inplace = True)


# 숫자형 변수를 연속형으로 취급하지 않고 category로 취급
for col in test.columns:
    train[col]=train[col].astype('category')
    test[col]=test[col].astype('category')

In [42]:
#Optuna용 Train셋
X = train.drop(['INVC_CONT'],axis=1)
y = train['INVC_CONT']
X_test = test.copy()

In [43]:
def objective(trial):
    param = {
        "random_state":42,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.05),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        "n_estimators":trial.suggest_int("n_estimators", 500, 5000),
        "max_depth":trial.suggest_int("max_depth", 4, 16),
        'random_strength' :trial.suggest_int('random_strength', 0, 100),
        "colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.4, 1.0),
        "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
    }
    X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.2)
    cat_features = range(X_test.shape[1])
    cat = CatBoostRegressor(**param)
    cat.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_valid,y_valid)],
            early_stopping_rounds=35,cat_features=cat_features,
            verbose=100)
    cat_pred = cat.predict(X_valid)
    rmse = np.sqrt(mean_squared_error(y_valid, cat_pred))
 
    return rmse

In [44]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'minimize',
    sampler = sampler,
)

study.optimize(objective, n_trials=10)
print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2024-03-28 14:23:08,638] A new study created in memory with name: cat_parameter_opt


0:	learn: 6.8455301	test: 6.8444973	test1: 6.3670641	best: 6.3670641 (0)	total: 330ms	remaining: 20m 51s
100:	learn: 6.5294859	test: 6.4568021	test1: 6.1712587	best: 6.1712587 (100)	total: 9.8s	remaining: 5m 58s
200:	learn: 6.2975867	test: 6.3183992	test1: 6.1099157	best: 6.1099039 (198)	total: 20.8s	remaining: 6m 11s
300:	learn: 6.2178044	test: 6.2982906	test1: 6.0998593	best: 6.0998593 (300)	total: 30s	remaining: 5m 47s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 6.099047354
bestIteration = 319

Shrink model to first 320 iterations.


[I 2024-03-28 14:23:44,622] Trial 0 finished with value: 6.0990473537165295 and parameters: {'learning_rate': 0.018272261776066247, 'bagging_temperature': 63.512210106407046, 'n_estimators': 3794, 'max_depth': 11, 'random_strength': 15, 'colsample_bylevel': 0.49359671220172163, 'l2_leaf_reg': 1.7519275289243016e-06, 'min_child_samples': 88, 'max_bin': 380, 'od_type': 'IncToDec'}. Best is trial 0 with value: 6.0990473537165295.


0:	learn: 6.9053516	test: 6.8859422	test1: 6.0936505	best: 6.0936505 (0)	total: 158ms	remaining: 3m 49s
100:	learn: 6.6171260	test: 6.4574115	test1: 5.8462296	best: 5.8460679 (99)	total: 6.39s	remaining: 1m 25s
200:	learn: 6.4048261	test: 6.3652476	test1: 5.6663164	best: 5.6636918 (187)	total: 11.4s	remaining: 1m 11s
300:	learn: 6.0300098	test: 6.4829543	test1: 5.5271259	best: 5.5271259 (300)	total: 17.4s	remaining: 1m 6s
400:	learn: 5.7792817	test: 6.5814173	test1: 5.4624072	best: 5.4572206 (375)	total: 23.9s	remaining: 1m 2s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 5.457220591
bestIteration = 375

Shrink model to first 376 iterations.


[I 2024-03-28 14:24:09,882] Trial 1 finished with value: 5.45722059094952 and parameters: {'learning_rate': 0.04763628595029446, 'bagging_temperature': 21.368329072358772, 'n_estimators': 1455, 'max_depth': 6, 'random_strength': 18, 'colsample_bylevel': 0.5825453457757226, 'l2_leaf_reg': 1.5747445384650815e-05, 'min_child_samples': 46, 'max_bin': 287, 'od_type': 'IncToDec'}. Best is trial 1 with value: 5.45722059094952.


...

0:	learn: 7.0098142	test: 7.0098142	test1: 5.6171659	best: 5.6171659 (0)	total: 51.2ms	remaining: 53.1s
100:	learn: 6.4850843	test: 6.3682805	test1: 5.1898840	best: 5.1890571 (92)	total: 11.9s	remaining: 1m 49s
200:	learn: 6.2625847	test: 6.3501865	test1: 5.1661159	best: 5.1637415 (183)	total: 23.1s	remaining: 1m 36s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 5.163741485
bestIteration = 183

Shrink model to first 184 iterations.


[I 2024-03-28 14:32:59,190] Trial 8 finished with value: 5.16374148476126 and parameters: {'learning_rate': 0.04169990777997927, 'bagging_temperature': 0.7742116473996251, 'n_estimators': 1038, 'max_depth': 13, 'random_strength': 76, 'colsample_bylevel': 0.7367663185416977, 'l2_leaf_reg': 2.3131305726837285e-05, 'min_child_samples': 52, 'max_bin': 357, 'od_type': 'IncToDec'}. Best is trial 8 with value: 5.16374148476126.


0:	learn: 6.7826095	test: 6.7818458	test1: 6.6406040	best: 6.6406040 (0)	total: 169ms	remaining: 9m 29s
100:	learn: 6.6673490	test: 6.5695070	test1: 6.5101422	best: 6.5101422 (100)	total: 7.74s	remaining: 4m 9s
200:	learn: 6.6102566	test: 6.4483657	test1: 6.4166871	best: 6.4166871 (200)	total: 13.7s	remaining: 3m 35s
300:	learn: 6.5752126	test: 6.3877042	test1: 6.3863571	best: 6.3863571 (300)	total: 20.4s	remaining: 3m 27s
400:	learn: 6.5450004	test: 6.3332659	test1: 6.3430061	best: 6.3430061 (400)	total: 27.1s	remaining: 3m 20s
500:	learn: 6.5203688	test: 6.3113372	test1: 6.3327242	best: 6.3321134 (496)	total: 33.9s	remaining: 3m 13s
600:	learn: 6.5030257	test: 6.2939659	test1: 6.3243921	best: 6.3243921 (600)	total: 40.6s	remaining: 3m 6s
700:	learn: 6.4859750	test: 6.2895567	test1: 6.3203468	best: 6.3203215 (697)	total: 47.6s	remaining: 3m
800:	learn: 6.4630615	test: 6.2756579	test1: 6.2992875	best: 6.2989887 (789)	total: 54.5s	remaining: 2m 54s
Stopped by overfitting detector  (35 i

[I 2024-03-28 14:33:56,069] Trial 9 finished with value: 6.298988697906698 and parameters: {'learning_rate': 0.011896326393433528, 'bagging_temperature': 0.0133572404119741, 'n_estimators': 3364, 'max_depth': 8, 'random_strength': 51, 'colsample_bylevel': 0.9445398843556558, 'l2_leaf_reg': 7.486273952174759e-06, 'min_child_samples': 44, 'max_bin': 427, 'od_type': 'IncToDec'}. Best is trial 8 with value: 5.16374148476126.


Best Score: 5.16374148476126
Best trial {'learning_rate': 0.04169990777997927, 'bagging_temperature': 0.7742116473996251, 'n_estimators': 1038, 'max_depth': 13, 'random_strength': 76, 'colsample_bylevel': 0.7367663185416977, 'l2_leaf_reg': 2.3131305726837285e-05, 'min_child_samples': 52, 'max_bin': 357, 'od_type': 'IncToDec'}


In [45]:
# 최적 하이퍼파라미터
cat_param = study.best_trial.params

In [46]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(train, train['INVC_CONT']):
    folds.append((train_idx,valid_idx))

In [47]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(train, train['INVC_CONT']):
    folds.append((train_idx,valid_idx))

random.seed(42)
cat_models={}

cat_features =range(X_test.shape[1])

for fold in range(10):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train = train.drop(['INVC_CONT'],axis=1).iloc[train_idx]
    X_valid = train.drop(['INVC_CONT'],axis=1).iloc[valid_idx]
    y_train = train['INVC_CONT'][train_idx].values
    y_valid = train['INVC_CONT'][valid_idx].values

    cat = CatBoostRegressor(**cat_param)
    cat.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid,y_valid)],
            early_stopping_rounds=35,cat_features=cat_features,
            verbose=100)
    cat_models[fold] = cat
    print(f'================================================================================\n\n')

====================================1============================================
0:	learn: 6.7620329	test: 6.7595631	test1: 6.6522580	best: 6.6522580 (0)	total: 106ms	remaining: 1m 49s
100:	learn: 6.2601972	test: 6.3751076	test1: 6.4185694	best: 6.4111990 (90)	total: 12.6s	remaining: 1m 57s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 6.397430977
bestIteration = 141

Shrink model to first 142 iterations.


====================================2============================================
0:	learn: 6.6234179	test: 6.6205735	test1: 7.7996692	best: 7.7996692 (0)	total: 142ms	remaining: 2m 27s
100:	learn: 6.1972868	test: 6.1117427	test1: 7.2062843	best: 7.2062843 (100)	total: 12.1s	remaining: 1m 52s
200:	learn: 5.9702972	test: 6.0208732	test1: 7.0550307	best: 7.0541032 (194)	total: 23.3s	remaining: 1m 37s
300:	learn: 5.6653044	test: 5.9428406	test1: 6.8129420	best: 6.8129420 (300)	total: 37.2s	remaining: 1m 31s
400:	learn: 4.9463628	test: 5.8000013	test1: 6.6868251	bes

In [49]:
for fold in range(10):
    submission.loc[:,'운송장_건수'] += cat_models[fold].predict(test)/10

In [52]:
submission.loc[submission.운송장_건수>30,'운송장_건수']=submission.loc[submission.운송장_건수>30,'운송장_건수']*4.8
submission.to_csv('./data/submission_0328.csv',index = False)

In [53]:
pd.read_csv("./data/submission.csv")

,index,운송장_건수
0,0,6.651738
1,1,5.951330
2,2,5.104412
3,3,4.519996
4,4,5.797217
...,...,...
7915,7915,5.169260
7916,7916,6.243764
7917,7917,3.711761
7918,7918,3.758050
